In [1]:
import numpy as np
import gudhi as gd

import plotly.graph_objects as go
from plotly.offline import iplot

import ipywidgets as widgets

In [2]:
def sample_from_torus(num_points, major_radius, minor_radius):
    # Generate random angles
    theta = 2 * np.pi * np.random.rand(num_points)
    phi = 2 * np.pi * np.random.rand(num_points)

    # Calculate x, y, z coordinates on the torus
    x = (major_radius + minor_radius * np.cos(phi)) * np.cos(theta)
    y = (major_radius + minor_radius * np.cos(phi)) * np.sin(theta)
    z = minor_radius * np.sin(phi)

    return x, y, z

num_points = 10000
major_radius = 5
minor_radius = 2

x, y, z = sample_from_torus(num_points, major_radius, minor_radius)

In [ ]:
fig = go.Figure(data=go.Scatter3d(x=x, y=y, z=z, mode='markers', 
                                  marker=dict(size=3, color='blue')))

fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
                  margin=dict(l=0, r=0, b=0, t=0), width=600, height=400)

In [4]:
torus = np.squeeze(np.dstack((x, y, z)))

In [ ]:
ac = gd.AlphaComplex(torus)
st = ac.create_simplex_tree()

n = st.dimension()

# Extract the 2-simplices in the Delaunay complex
triangles = [s for s in st.get_skeleton(n) if len(s[0]) == 3]

alpha = widgets.FloatSlider(
    value = 0.15,
    min = 0.015,
    max = 0.3,
    step = 0.0001,
    description = 'Alpha:', 
    readout_format = '.4f',
    layout=widgets.Layout(width='600px')
)
mesh = go.Mesh3d(
    x = torus[:, 0], 
    y = torus[:, 1], 
    z = torus[:, 2], 
    i = torus[:, 0], 
    j = torus[:, 1], 
    k = torus[:, 2]
)
fig = go.FigureWidget(
    data = mesh, 
    layout = go.Layout(
        width=600,
        height=400
    )
)
def view_triangulation(alpha):
    # Filter to get the 2-simplices in the alpha complex
    alpha_triangles = np.array([s[0] for s in triangles if s[1] < alpha])
    fig.data[0].i = alpha_triangles[:, 0]
    fig.data[0].j = alpha_triangles[:, 1]
    fig.data[0].k = alpha_triangles[:, 2]
    iplot(fig)

fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
                  margin=dict(l=0, r=0, b=0, t=0), width=600, height=400)

widgets.interact(view_triangulation, alpha=alpha)

In [ ]:
r = widgets.FloatSlider(
    value = 0.3,
    min = 0.1,
    max = 0.6,
    step = 0.0001,
    description = 'r:', 
    readout_format = '.4f',
    layout=widgets.Layout(width='600px')
)
mesh = go.Mesh3d(
    x = torus[:, 0], 
    y = torus[:, 1], 
    z = torus[:, 2], 
    i = torus[:, 0], 
    j = torus[:, 1], 
    k = torus[:, 2]
)
fig = go.FigureWidget(
    data = mesh, 
    layout = go.Layout(
        width=600,
        height=400
    )
)
def view_triangulation(r):
    # Create the Rips complex
    rc = gd.RipsComplex(points=torus, max_edge_length=2*r)
    st = rc.create_simplex_tree(max_dimension=2)

    # Extract the 2-simplices
    triangles = np.array([s for s, _ in st.get_simplices() if len(s) == 3])
    
    fig.data[0].i = triangles[:, 0]
    fig.data[0].j = triangles[:, 1]
    fig.data[0].k = triangles[:, 2]
    iplot(fig)

fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
                  margin=dict(l=0, r=0, b=0, t=0), width=600, height=400)

widgets.interact(view_triangulation, r=r)